# Typo Detector with OpenVINO™

Typo detection in AI is a process of identifying and correcting typographical errors in text data using machine learning algorithms. The goal of typo detection is to improve the accuracy, readability, and usability of text by identifying and indicating mistakes made during the writing process. To detect typos, AI-based typo detectors use various techniques, such as natural language processing (NLP), machine learning (ML), and deep learning (DL).

A typo detector takes a sentence as an input and identify all typographical errors such as misspellings and homophone errors.

In [1]:
%pip install -q "diffusers>=0.17.1" "openvino>=2023.1.0" "nncf>=2.5.0" "gradio" "onnx>=1.11.0" "onnxruntime>=1.14.0" "transformers[torch]>=4.31.0"
%pip install -q "git+https://github.com/huggingface/optimum-intel.git"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.5/37.5 MB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 56.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 54.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 59.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.7/120.7 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### Imports [$\Uparrow$](#Table-of-content:)


In [2]:
from transformers import AutoConfig, AutoTokenizer, AutoModelForTokenClassification, pipeline
from pathlib import Path
import numpy as np
import re
from typing import List, Dict
import time

### Methods [$\Uparrow$](#Table-of-content:)

The notebook provides two methods to run the inference of typo detector with OpenVINO runtime, so that you can experience both calling the API of Optimum with OpenVINO Runtime included, and loading models in other frameworks, converting them to OpenVINO IR format, and running inference with OpenVINO Runtime.

##### 1. Using the [Hugging Face Optimum](https://huggingface.co/docs/optimum/index) library [$\Uparrow$](#Table-of-content:)
The Hugging Face Optimum API is a high-level API that allows us to convert models from the Hugging Face Transformers library to the OpenVINO™ IR format. Compiled models in OpenVINO IR format can be loaded using Optimum. Optimum allows the use of optimization on targeted hardware.

##### 2. Converting the model to OpenVINO IR [$\Uparrow$](#Table-of-content:)
The Pytorch model is converted to [OpenVINO IR format](https://docs.openvino.ai/latest/openvino_ir.html). This method provides much more insight to how to set up a pipeline from model loading to model converting, compiling and running inference with OpenVINO, so that you could conveniently use OpenVINO to optimize and accelerate inference for other deep-learning models. The optimization of targeted hardware is also used here.


The following table summarizes the major differences between the two methods

</br>

| Method 1                                                            | Method 2                                                           |
| ------------------------------------------------------------------- | ------------------------------------------------------------------ |
| Load models from Optimum, an extension of transformers              | Load model from transformers                                       |
| Load the model in OpenVINO IR format on the fly                     | Convert to OpenVINO IR                                             |
| Load the compiled model by default                                  | Compile the OpenVINO IR and run inference with OpenVINO Runtime    |
| Pipeline is created to run inference with OpenVINO Runtime          | Manually run inference.                                            |


### Select inference device [$\Uparrow$](#Table-of-content:)

select device from dropdown list for running inference using OpenVINO

In [3]:
import ipywidgets as widgets
import openvino as ov

core = ov.Core()

device = widgets.Dropdown(
    options=core.available_devices + ["AUTO"],
    value='AUTO',
    description='Device:',
    disabled=False,
)

device

Dropdown(description='Device:', index=1, options=('CPU', 'AUTO'), value='AUTO')

### 1. Hugging Face Optimum Intel library [$\Uparrow$](#Table-of-content:)

For this method, we need to install the `Hugging Face Optimum Intel library` accelerated by OpenVINO integration.

Optimum Intel can be used to load optimized models from the [Hugging Face Hub](https://huggingface.co/docs/optimum/intel/hf.co/models) and create pipelines to run an inference with OpenVINO Runtime using Hugging Face APIs. The Optimum Inference models are API compatible with Hugging Face Transformers models.  This means we need just replace `AutoModelForXxx` class with the corresponding `OVModelForXxx` class.

Import required model class

In [4]:
from optimum.intel.openvino import OVModelForTokenClassification

INFO:nncf:NNCF initialized successfully. Supported frameworks detected: torch, tensorflow, onnx, openvino


##### Load the model [$\Uparrow$](#Table-of-content:)

From the `OVModelForTokenCLassification` class we will import the relevant pre-trained model. To load a Transformers model and convert it to the OpenVINO format on-the-fly, we set `export=True` when loading your model.

In [5]:
# The pretrained model we are using
model_id = "m3hrdadfi/typo-detector-distilbert-en"

model_dir = Path("optimum_model")

# Save the model to the path if not existing
if model_dir.exists():
    model = OVModelForTokenClassification.from_pretrained(model_dir, device=device.value)
else:
    model = OVModelForTokenClassification.from_pretrained(model_id, export=True, device=device.value)
    model.save_pretrained(model_dir)

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

Framework not specified. Using pt to export to ONNX.


pytorch_model.bin:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/365 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Using the export variant default. Available variants are:
    - default: The default ONNX variant.
Using framework PyTorch: 2.1.0+cu118
/usr/local/lib/python3.10/dist-packages/nncf/torch/dynamic_graph/wrappers.py:75: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  op1 = operator(*args, **kwargs)
Compiling the model to AUTO ...


##### Load the tokenizer [$\Uparrow$](#Table-of-content:)

Text Preprocessing cleans the text-based input data so it can be fed into the model. Tokenization splits paragraphs and sentences into smaller units that can be more easily assigned meaning. It involves cleaning the data and assigning tokens or IDs to the words, so they are represented in a vector space where similar words have similar vectors. This helps the model understand the context of a sentence. We're making use of an [AutoTokenizer](https://huggingface.co/docs/transformers/main_classes/tokenizer) from Hugging Face, which is essentially a pretrained tokenizer.

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

Then we use the inference pipeline for `token-classification` task. You can find more information about usage Hugging Face inference pipelines in this [tutorial](https://huggingface.co/docs/transformers/pipeline_tutorial)

In [7]:
nlp = pipeline('token-classification', model=model, tokenizer=tokenizer, aggregation_strategy="average")

Function to find typos in a sentence and write them to the terminal

In [8]:
def show_typos(sentence: str):
    """
    Detect typos from the given sentence.
    Writes both the original input and typo-tagged version to the terminal.

    Arguments:
    sentence -- Sentence to be evaluated (string)
    """

    typos = [sentence[r["start"]: r["end"]] for r in nlp(sentence)]

    detected = sentence
    for typo in typos:
        detected = detected.replace(typo, f'<i>{typo}</i>')

    print("[Input]: ", sentence)
    print("[Detected]: ", detected)
    print("-" * 130)

Let's run a demo using the Hugging Face Optimum API.

In [9]:
sentences = [
    "He had also stgruggled with addiction during his time in Congress .",
    "The review thoroughla assessed all aspects of JLENS SuR and CPG esign maturit and confidence .",
    "Letterma also apologized two his staff for the satyation .",
    "Vincent Jay had earlier won France 's first gold in gthe 10km biathlon sprint .",
    "It is left to the directors to figure out hpw to bring the stry across to tye audience .",
    "I wnet to the park yestreday to play foorball with my fiends, but it statred to rain very hevaily and we had to stop.",
    "My faorite restuarant servs the best spahgetti in the town, but they are always so buzy that you have to make a resrvation in advnace.",
    "I was goig to watch a mvoie on Netflx last night, but the straming was so slow that I decided to cancled my subscrpition.",
    "My freind and I went campign in the forest last weekend and saw a beutiful sunst that was so amzing it took our breth away.",
    "I  have been stuying for my math exam all week, but I'm stil not very confidet that I will pass it, because there are so many formuals to remeber."
]

start = time.time()

for sentence in sentences:
    show_typos(sentence)

print(f"Time elapsed: {time.time() - start}")

[Input]:  He had also stgruggled with addiction during his time in Congress .
[Detected]:  He had also <i>stgruggled</i> with addiction during his time in Congress .
----------------------------------------------------------------------------------------------------------------------------------
[Input]:  The review thoroughla assessed all aspects of JLENS SuR and CPG esign maturit and confidence .
[Detected]:  The review <i>thoroughla</i> assessed all aspects of JLENS SuR and CPG <i>esign maturit</i> and confidence .
----------------------------------------------------------------------------------------------------------------------------------
[Input]:  Letterma also apologized two his staff for the satyation .
[Detected]:  <i>Letterma</i> also apologized <i>two</i> his staff for the <i>satyation</i> .
----------------------------------------------------------------------------------------------------------------------------------
[Input]:  Vincent Jay had earlier won France 's firs

### 2. Converting the model to OpenVINO IR [$\Uparrow$](#Table-of-content:)


##### Load the Pytorch model [$\Uparrow$](#Table-of-content:)

Use the `AutoModelForTokenClassification` class to load the pretrained pytorch model.

In [10]:
model_id = "m3hrdadfi/typo-detector-distilbert-en"
model_dir = Path("pytorch_model")

tokenizer = AutoTokenizer.from_pretrained(model_id)
config = AutoConfig.from_pretrained(model_id)

# Save the model to the path if not existing
if model_dir.exists():
    model = AutoModelForTokenClassification.from_pretrained(model_dir)
else:
    model = AutoModelForTokenClassification.from_pretrained(model_id, config=config)
    model.save_pretrained(model_dir)

##### Converting to OpenVINO IR [$\Uparrow$](#Table-of-content:)

In [11]:
ov_model_path = Path(model_dir) / "typo_detect.xml"

dummy_model_input = tokenizer("This is a sample", return_tensors="pt")
ov_model = ov.convert_model(model, example_input=dict(dummy_model_input))
ov.save_model(ov_model, ov_model_path)

##### Inference [$\Uparrow$](#Table-of-content:)

OpenVINO™ Runtime Python API is used to compile the model in OpenVINO IR format. The Core class from the `openvino` module is imported first. This class provides access to the OpenVINO Runtime API. The `core` object, which is an instance of the `Core` class, represents the API and it is used to compile the model. The output layer is extracted from the compiled model as it is needed for inference.

In [12]:
compiled_model = core.compile_model(ov_model, device.value)
output_layer = compiled_model.output(0)

### Helper Functions [$\Uparrow$](#Table-of-content:)


In [13]:
def token_to_words(tokens: List[str]) -> Dict[str, int]:
    """
    Maps the list of tokens to words in the original text.
    Built on the feature that tokens starting with '##' is attached to the previous token as tokens derived from the same word.

    Arguments:
    tokens -- List of tokens

    Returns:
    map_to_words -- Dictionary mapping tokens to words in original text
    """

    word_count = -1
    map_to_words = {}
    for token in tokens:
        if token.startswith('##'):
            map_to_words[token] = word_count
            continue
        word_count += 1
        map_to_words[token] = word_count
    return map_to_words

In [14]:
def infer(input_text: str) -> Dict[np.ndarray, np.ndarray]:
    """
    Creating a generic inference function to read the input and infer the result

    Arguments:
    input_text -- The text to be infered (String)

    Returns:
    result -- Resulting list from inference
    """

    tokens = tokenizer(
        input_text,
        return_tensors="np",
    )
    inputs = dict(tokens)
    result = compiled_model(inputs)[output_layer]
    return result

In [15]:
def get_typo_indexes(result: Dict[np.ndarray, np.ndarray], map_to_words: Dict[str, int], tokens: List[str]) -> List[int]:
    """
    Given results from the inference and tokens-map-to-words, identifies the indexes of the words with typos.

    Arguments:
    result -- Result from inference (tensor)
    map_to_words -- Dictionary mapping tokens to words (Dictionary)

    Results:
    wrong_words -- List of indexes of words with typos
    """

    wrong_words = []
    c = 0
    result_list = result[0][1:-1]
    for i in result_list:
        prob = np.argmax(i)
        if prob == 1:
            if map_to_words[tokens[c]] not in wrong_words:
                wrong_words.append(map_to_words[tokens[c]])
        c += 1
    return wrong_words

In [16]:
def sentence_split(sentence: str) -> List[str]:
    """
    Split the sentence into words and characters

    Arguments:
    sentence - Sentence to be split (string)

    Returns:
    splitted -- List of words and characters
    """

    splitted = re.split("([',. ])",sentence)
    splitted = [x for x in splitted if x != " " and x != ""]
    return splitted

In [17]:
def show_typos(sentence: str):
    """
    Detect typos from the given sentence.
    Writes both the original input and typo-tagged version to the terminal.

    Arguments:
    sentence -- Sentence to be evaluated (string)
    """

    tokens = tokenizer.tokenize(sentence)
    map_to_words = token_to_words(tokens)
    result = infer(sentence)
    typo_indexes = get_typo_indexes(result,map_to_words, tokens)

    sentence_words = sentence_split(sentence)

    typos = [sentence_words[i] for i in typo_indexes]

    detected = sentence
    for typo in typos:
        detected = detected.replace(typo, f'<i>{typo}</i>')

    print("   [Input]: ", sentence)
    print("[Detected]: ", detected)
    print("-" * 130)

Let's run a demo using the converted OpenVINO IR model.

In [18]:
sentences = [
    "He had also stgruggled with addiction during his time in Congress .",
    "The review thoroughla assessed all aspects of JLENS SuR and CPG esign maturit and confidence .",
    "Letterma also apologized two his staff for the satyation .",
    "Vincent Jay had earlier won France 's first gold in gthe 10km biathlon sprint .",
    "It is left to the directors to figure out hpw to bring the stry across to tye audience .",
    "I wnet to the park yestreday to play foorball with my fiends, but it statred to rain very hevaily and we had to stop.",
    "My faorite restuarant servs the best spahgetti in the town, but they are always so buzy that you have to make a resrvation in advnace.",
    "I was goig to watch a mvoie on Netflx last night, but the straming was so slow that I decided to cancled my subscrpition.",
    "My freind and I went campign in the forest last weekend and saw a beutiful sunst that was so amzing it took our breth away.",
    "I  have been stuying for my math exam all week, but I'm stil not very confidet that I will pass it, because there are so many formuals to remeber."
]

start = time.time()

for sentence in sentences:
    show_typos(sentence)

print(f"Time elapsed: {time.time() - start}")

   [Input]:  He had also stgruggled with addiction during his time in Congress .
[Detected]:  He had also <i>stgruggled</i> with addiction during his time in Congress .
----------------------------------------------------------------------------------------------------------------------------------
   [Input]:  The review thoroughla assessed all aspects of JLENS SuR and CPG esign maturit and confidence .
[Detected]:  The review <i>thoroughla</i> assessed all aspects of JLENS SuR and CPG <i>esign</i> <i>maturit</i> and confidence .
----------------------------------------------------------------------------------------------------------------------------------
   [Input]:  Letterma also apologized two his staff for the satyation .
[Detected]:  <i>Letterma</i> also apologized <i>two</i> his staff for the <i>satyation</i> .
----------------------------------------------------------------------------------------------------------------------------------
   [Input]:  Vincent Jay had earlier

In [19]:
sentences = [
    "Sophia said, Get out of the rain!",
    "Once the train arrives we will be on are way.",
    "Gas in Alaska is the most cheapest gas in all the world. ",
    "When I turn forty five I would have been a coal miner for twenty two years.",
    "Apollo 11 was the first manned mission to land on the moon.",
    "The cashere gave me two hundred and sixteen dollars sixty two cents."
]

start = time.time()

for sentence in sentences:
    show_typos(sentence)

print(f"Time elapsed: {time.time() - start}")

   [Input]:  Sophia said, Get out of the rain!
[Detected]:  Sophia said, Get out of the rain!
----------------------------------------------------------------------------------------------------------------------------------
   [Input]:  Once the train arrives we will be on are way.
[Detected]:  Once the train arrives we will be on <i>are</i> way.
----------------------------------------------------------------------------------------------------------------------------------
   [Input]:  Gas in Alaska is the most cheapest gas in all the world. 
[Detected]:  Gas in Alaska is the most cheapest gas in all the world. 
----------------------------------------------------------------------------------------------------------------------------------
   [Input]:  When I turn forty five I would have been a coal miner for twenty two years.
[Detected]:  When I turn forty five I would have been a coal miner for twenty two years.
--------------------------------------------------------------------